<a href="https://colab.research.google.com/github/tivon-x/all-rag-techniques/blob/main/13_self_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 自我 RAG：一种动态的 RAG 方法

在本笔记本中，我实现了自我 RAG，这是一种先进的 RAG 系统，能够动态决定何时以及如何使用检索到的信息。与传统的 RAG 方法不同，自我 RAG 在检索和生成过程中引入了反思点，从而产生更高质量且更可靠的响应。

## 自我 RAG 的关键组件

1. **检索决策**：确定对于给定的查询是否需要检索
2. **文档检索**：在需要时检索可能相关的文档
3. **相关性评估**：评估每篇检索到的文档的相关性
4. **响应生成**：基于相关上下文创建响应
5. **支持评估**：评估响应是否基于上下文合理生成
6. **效用评估**：对生成的响应的整体有用性进行评分

<div style="text-align: center;">

<img src="https://github.com/tivon-x/all-rag-techniques/blob/main/images/self_rag.svg?raw=1" alt="Self RAG" style="width:80%; height:auto;">
</div>

## 环境配置

In [1]:
# fitz库需要从pymudf那里安装
%pip install --quiet --force-reinstall pymupdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 37.2 MB/s eta 0:00:00


In [2]:
import os
import numpy as np
import json
import fitz
from openai import OpenAI
import re

## 抽取文本

In [3]:
def extract_text_from_pdf(pdf_path):
    """
    Extracts text from a PDF file.

    Args:
    pdf_path (str): Path to the PDF file.

    Returns:
    str: Extracted text from the PDF.
    """
    # Open the PDF file
    mypdf = fitz.open(pdf_path)
    all_text = ""  # Initialize an empty string to store the extracted text

    # Iterate through each page in the PDF
    for page_num in range(mypdf.page_count):
        page = mypdf[page_num]  # Get the page
        text = page.get_text("text")  # Extract text from the page
        all_text += text  # Append the extracted text to the all_text string

    return all_text  # Return the extracted text

## 分块

In [4]:
def chunk_text(text, n, overlap):
    """
    Chunks the given text into segments of n characters with overlap.

    Args:
    text (str): The text to be chunked.
    n (int): The number of characters in each chunk.
    overlap (int): The number of overlapping characters between chunks.

    Returns:
    List[str]: A list of text chunks.
    """
    chunks = []  # Initialize an empty list to store the chunks

    # Loop through the text with a step size of (n - overlap)
    for i in range(0, len(text), n - overlap):
        # Append a chunk of text from index i to i + n to the chunks list
        chunks.append(text[i:i + n])

    return chunks  # Return the list of text chunks

## OpenAI Client

In [5]:
# colab环境
from google.colab import userdata
# 使用火山引擎
api_key = userdata.get("ARK_API_KEY")
base_url = userdata.get("ARK_BASE_URL")

In [6]:
model_name = "doubao-lite-128k-240828"
embedding_model = "doubao-embedding-text-240715"

In [8]:
client = OpenAI(
    base_url=base_url,
    api_key=api_key
)

## 向量数据库

In [9]:
class SimpleVectorStore:
    """
    A simple vector store implementation using NumPy.
    """
    def __init__(self):
        """
        Initialize the vector store.
        """
        self.vectors = []  # List to store embedding vectors
        self.texts = []  # List to store original texts
        self.metadata = []  # List to store metadata for each text

    def add_item(self, text, embedding, metadata=None):
        """
        Add an item to the vector store.

        Args:
        text (str): The original text.
        embedding (List[float]): The embedding vector.
        metadata (dict, optional): Additional metadata.
        """
        self.vectors.append(np.array(embedding))  # Convert embedding to numpy array and add to vectors list
        self.texts.append(text)  # Add the original text to texts list
        self.metadata.append(metadata or {})  # Add metadata to metadata list, default to empty dict if None

    def similarity_search(self, query_embedding, k=5, filter_func=None):
        """
        Find the most similar items to a query embedding.

        Args:
        query_embedding (List[float]): Query embedding vector.
        k (int): Number of results to return.
        filter_func (callable, optional): Function to filter results.

        Returns:
        List[Dict]: Top k most similar items with their texts and metadata.
        """
        if not self.vectors:
            return []  # Return empty list if no vectors are stored

        # Convert query embedding to numpy array
        query_vector = np.array(query_embedding)

        # Calculate similarities using cosine similarity
        similarities = []
        for i, vector in enumerate(self.vectors):
            # Apply filter if provided
            if filter_func and not filter_func(self.metadata[i]):
                continue

            # Calculate cosine similarity
            similarity = np.dot(query_vector, vector) / (np.linalg.norm(query_vector) * np.linalg.norm(vector))
            similarities.append((i, similarity))  # Append index and similarity score

        # Sort by similarity (descending)
        similarities.sort(key=lambda x: x[1], reverse=True)

        # Return top k results
        results = []
        for i in range(min(k, len(similarities))):
            idx, score = similarities[i]
            results.append({
                "text": self.texts[idx],  # Add the text
                "metadata": self.metadata[idx],  # Add the metadata
                "similarity": score  # Add the similarity score
            })

        return results  # Return the list of top k results

## 向量嵌入

In [10]:
def create_embeddings(text, model = None, batch_size = 10):
    """
    Creates embeddings for the given text.

    Args:
    text (str or List[str]): The input text(s) for which embeddings are to be created.
    model (str): The model to be used for creating embeddings.
    batch_size (int): batch size

    Returns:
    List[float] or List[List[float]]: The embedding vector(s).
    """
    if not model:
      model = embedding_model

    # Convert single string to list for uniform processing
    input_text = text if isinstance(text, list) else [text]

    all_embeddings = []

    for i in range(0, len(input_text), batch_size):
      batch = input_text[i : i + batch_size]
      # Create embeddings for the batch using the specified model
      response = client.embeddings.create(
          model=model,
          input=batch
      )
      all_embeddings.extend(item.embedding for item in response.data)

    return all_embeddings if isinstance(text, list) else all_embeddings[0]

## 文档处理流程

In [11]:
def process_document(pdf_path, chunk_size=1000, chunk_overlap=200):
    """
    Process a document for Self-RAG.

    Args:
        pdf_path (str): Path to the PDF file.
        chunk_size (int): Size of each chunk in characters.
        chunk_overlap (int): Overlap between chunks in characters.

    Returns:
        SimpleVectorStore: A vector store containing document chunks and their embeddings.
    """
    # Extract text from the PDF file
    print("Extracting text from PDF...")
    extracted_text = extract_text_from_pdf(pdf_path)

    # Chunk the extracted text
    print("Chunking text...")
    chunks = chunk_text(extracted_text, chunk_size, chunk_overlap)
    print(f"Created {len(chunks)} text chunks")

    # Create embeddings for each chunk
    print("Creating embeddings for chunks...")
    chunk_embeddings = create_embeddings(chunks)

    # Initialize the vector store
    store = SimpleVectorStore()

    # Add each chunk and its embedding to the vector store
    for i, (chunk, embedding) in enumerate(zip(chunks, chunk_embeddings)):
        store.add_item(
            text=chunk,
            embedding=embedding,
            metadata={"index": i, "source": pdf_path}
        )

    print(f"Added {len(chunks)} chunks to the vector store")
    return store

## Self-RAG 组件
### 1. 检索决策

In [13]:
def determine_if_retrieval_needed(query):
    """
    确定对于给定的查询是否需要检索。

    Args:
        query (str): User query

    Returns:
        bool: True if retrieval is needed, False otherwise
    """
    # System prompt, 指导 AI 如何确定是否需要检索
    system_prompt = """You are an AI assistant that determines if retrieval is necessary to answer a query.
    For factual questions, specific information requests, or questions about events, people, or concepts, answer "Yes".
    For opinions, hypothetical scenarios, or simple queries with common knowledge, answer "No".
    Answer with ONLY "Yes" or "No"."""

    # User prompt containing the query
    user_prompt = f"Query: {query}\n\nIs retrieval necessary to answer this query accurately?"

    # Generate response from the model
    response = client.chat.completions.create(
        model= model_name,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0
    )

    # Extract the answer from the model's response and convert to lowercase
    answer = response.choices[0].message.content.strip().lower()

    # Return True if the answer contains "yes", otherwise return False
    return "yes" in answer

### 2. 相关性评估

In [14]:
def evaluate_relevance(query, context):
    """
    评估上下文与查询的相关性。

    Args:
        query (str): User query
        context (str): Context text

    Returns:
        str: 'relevant' or 'irrelevant'
    """
    # System prompt，指导 llm 评估文档相关性
    system_prompt = """You are an AI assistant that determines if a document is relevant to a query.
    Consider whether the document contains information that would be helpful in answering the query.
    Answer with ONLY "Relevant" or "Irrelevant"."""

    # you can truncate context if it is too long to avoid exceeding token limits
    max_context_length = 2000
    if len(context) > max_context_length:
        context = context[:max_context_length] + "... [truncated]"

    # User prompt containing the query and the document content
    user_prompt = f"""Query: {query}
    Document content:
    {context}

    Is this document relevant to the query? Answer with ONLY "Relevant" or "Irrelevant".
    """

    # Generate response from the model
    response = client.chat.completions.create(
        model=model_name,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0
    )

    # Extract the answer from the model's response and convert to lowercase
    answer = response.choices[0].message.content.strip().lower()

    return answer  # Return the relevance evaluation

### 3. 支持评估

In [16]:
def assess_support(response, context):
    """
    评估响应是否基于上下文合理生成。

    Args:
        response (str): Generated response
        context (str): Context text

    Returns:
        str: 'fully supported', 'partially supported', or 'no support'
    """
    # System prompt，指导模型如何评估支持度
    system_prompt = """You are an AI assistant that determines if a response is supported by the given context.
    Evaluate if the facts, claims, and information in the response are backed by the context.
    Answer with ONLY one of these three options:
    - "Fully supported": All information in the response is directly supported by the context.
    - "Partially supported": Some information in the response is supported by the context, but some is not.
    - "No support": The response contains significant information not found in or contradicting the context.
    """

    # Truncate context if it is too long to avoid exceeding token limits
    max_context_length = 2000
    if len(context) > max_context_length:
        context = context[:max_context_length] + "... [truncated]"

    # User prompt containing the context and the response to be evaluated
    user_prompt = f"""Context:
    {context}

    Response:
    {response}

    How well is this response supported by the context? Answer with ONLY "Fully supported", "Partially supported", or "No support".
    """

    # Generate response from the model
    response = client.chat.completions.create(
        model=model_name,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0
    )

    # Extract the answer from the model's response and convert to lowercase
    answer = response.choices[0].message.content.strip().lower()

    return answer  # Return the support assessment

### 4. 效用评估

In [17]:
def rate_utility(query, response):
    """
    对查询的响应的有用性进行评分。

    Args:
        query (str): User query
        response (str): Generated response

    Returns:
        int: Utility rating from 1 to 5
    """
    # System prompt，指导模型如何评估效用
    system_prompt = """You are an AI assistant that rates the utility of a response to a query.
    Consider how well the response answers the query, its completeness, correctness, and helpfulness.
    Rate the utility on a scale from 1 to 5, where:
    - 1: Not useful at all
    - 2: Slightly useful
    - 3: Moderately useful
    - 4: Very useful
    - 5: Exceptionally useful
    Answer with ONLY a single number from 1 to 5."""

    # User prompt containing the query and the response to be rated
    user_prompt = f"""Query: {query}
    Response:
    {response}

    Rate the utility of this response on a scale from 1 to 5:"""

    # Generate the utility rating using the OpenAI client
    response = client.chat.completions.create(
        model=model_name,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0
    )

    # Extract the rating from the model's response
    rating = response.choices[0].message.content.strip()

    # Extract just the number from the rating
    rating_match = re.search(r'[1-5]', rating)
    if rating_match:
        return int(rating_match.group())  # Return the extracted rating as an integer

    return 3  # Default to middle rating if parsing fails

## 响应生成

In [19]:
def generate_response(query, context=None):
    """
    Generates a response based on the query and optional context.

    Args:
        query (str): User query
        context (str, optional): Context text

    Returns:
        str: Generated response
    """
    # System prompt to instruct the AI on how to generate a helpful response
    system_prompt = """You are a helpful AI assistant. Provide a clear, accurate, and informative response to the query."""

    # Create the user prompt based on whether context is provided
    if context:
        user_prompt = f"""Context:
        {context}

        Query: {query}

        Please answer the query based on the provided context.
        """
    else:
        user_prompt = f"""Query: {query}

        Please answer the query to the best of your ability."""

    # Generate the response using the OpenAI client
    response = client.chat.completions.create(
        model=model_name,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0.2
    )

    # Return the generated response text
    return response.choices[0].message.content.strip()

## Self-RAG pipeline 实现

In [23]:
def self_rag(query, vector_store, top_k=3):
    """
    Implements the complete Self-RAG pipeline.

    Args:
        query (str): User query
        vector_store (SimpleVectorStore): Vector store containing document chunks
        top_k (int): Number of documents to retrieve initially

    Returns:
        dict: Results including query, response, and metrics from the Self-RAG process
    """
    print(f"\n=== Starting Self-RAG for query: {query} ===\n")

    # Step 1: Determine if retrieval is necessary
    print("Step 1: Determining if retrieval is necessary...")
    retrieval_needed = determine_if_retrieval_needed(query)
    print(f"Retrieval needed: {retrieval_needed}")

    # Initialize metrics to track the Self-RAG process
    metrics = {
        "retrieval_needed": retrieval_needed,
        "documents_retrieved": 0,
        "relevant_documents": 0,
        "response_support_ratings": [],
        "utility_ratings": []
    }

    best_response = None
    best_score = -1

    if retrieval_needed:
        # Step 2: Retrieve documents
        print("\nStep 2: Retrieving relevant documents...")
        query_embedding = create_embeddings(query)
        results = vector_store.similarity_search(query_embedding, k=top_k)
        metrics["documents_retrieved"] = len(results)
        print(f"Retrieved {len(results)} documents")

        # Step 3: Evaluate relevance of each document
        print("\nStep 3: Evaluating document relevance...")
        relevant_contexts = []

        for i, result in enumerate(results):
            context = result["text"]
            relevance = evaluate_relevance(query, context)
            print(f"Document {i+1} relevance: {relevance}")

            if relevance == "relevant":
                relevant_contexts.append(context)

        metrics["relevant_documents"] = len(relevant_contexts)
        print(f"Found {len(relevant_contexts)} relevant documents")

        if relevant_contexts:
            # Step 4: Process each relevant context
            print("\nStep 4: Processing relevant contexts...")
            for i, context in enumerate(relevant_contexts):
                print(f"\n ============== Processing context {i+1}/{len(relevant_contexts)}...")

                # Generate response based on the context
                print("Generating response...")
                response = generate_response(query, context)

                # Assess how well the response is supported by the context
                print("Assessing support...")
                support_rating = assess_support(response, context)
                print(f"Support rating: {support_rating}")
                metrics["response_support_ratings"].append(support_rating)

                # Rate the utility of the response
                print("Rating utility...")
                utility_rating = rate_utility(query, response)
                print(f"Utility rating: {utility_rating}/5")
                metrics["utility_ratings"].append(utility_rating)

                # Calculate overall score (higher for better support and utility)
                support_score = {
                    "fully supported": 3,
                    "partially supported": 1,
                    "no support": 0
                }.get(support_rating, 0)

                overall_score = support_score * 5 + utility_rating
                print(f"Overall score: {overall_score}")

                # Keep track of the best response
                if overall_score > best_score:
                    best_response = response
                    best_score = overall_score
                    print("New best response found!")

        # If no relevant contexts were found or all responses scored poorly
        if not relevant_contexts or best_score <= 0:
            print("\nNo suitable context found or poor responses, generating without retrieval...")
            best_response = generate_response(query)
    else:
        # No retrieval needed, generate directly
        print("\nNo retrieval needed, generating response directly...")
        best_response = generate_response(query)

    # Final metrics
    metrics["best_score"] = best_score
    metrics["used_retrieval"] = retrieval_needed and best_score > 0

    print("\n=== Self-RAG Completed ===")

    return {
        "query": query,
        "response": best_response,
        "metrics": metrics
    }

## 运行

In [25]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [26]:
def run_self_rag_example():
    """
    Demonstrates the complete Self-RAG system with examples.
    """
    # Process document
    pdf_path = "./drive/MyDrive/colab_data/AI_Information.pdf"  # Path to the PDF document
    print(f"Processing document: {pdf_path}")
    vector_store = process_document(pdf_path)  # Process the document and create a vector store

    # Example 1: Query likely needing retrieval
    query1 = "What are the main ethical concerns in AI development?"
    print("\n" + "="*80)
    print(f"EXAMPLE 1: {query1}")
    result1 = self_rag(query1, vector_store)  # Run Self-RAG for the first query
    print("\nFinal response:")
    print(result1["response"])  # Print the final response for the first query
    print("\nMetrics:")
    print(json.dumps(result1["metrics"], indent=2))  # Print the metrics for the first query

    # Example 2: Query likely not needing retrieval
    query2 = "Can you write a short poem about artificial intelligence?"
    print("\n" + "="*80)
    print(f"EXAMPLE 2: {query2}")
    result2 = self_rag(query2, vector_store)  # Run Self-RAG for the second query
    print("\nFinal response:")
    print(result2["response"])  # Print the final response for the second query
    print("\nMetrics:")
    print(json.dumps(result2["metrics"], indent=2))  # Print the metrics for the second query

    # Example 3: Query with some relevance to document but requiring additional knowledge
    query3 = "How might AI impact economic growth in developing countries?"
    print("\n" + "="*80)
    print(f"EXAMPLE 3: {query3}")
    result3 = self_rag(query3, vector_store)  # Run Self-RAG for the third query
    print("\nFinal response:")
    print(result3["response"])  # Print the final response for the third query
    print("\nMetrics:")
    print(json.dumps(result3["metrics"], indent=2))  # Print the metrics for the third query

    return {
        "example1": result1,
        "example2": result2,
        "example3": result3
    }

In [33]:
run_self_rag_example()

Processing document: ./drive/MyDrive/colab_data/AI_Information.pdf
Extracting text from PDF...
Chunking text...
Created 42 text chunks
Creating embeddings for chunks...
Added 42 chunks to the vector store

EXAMPLE 1: What are the main ethical concerns in AI development?

=== Starting Self-RAG for query: What are the main ethical concerns in AI development? ===

Step 1: Determining if retrieval is necessary...
Retrieval needed: True

Step 2: Retrieving relevant documents...
Retrieved 3 documents

Step 3: Evaluating document relevance...
Document 1 relevance: relevant
Document 2 relevance: relevant
Document 3 relevance: relevant
Found 3 relevant documents

Step 4: Processing relevant contexts...

 ============== Processing context 1/3...
Generating response...
Assessing support...
Support rating: fully supported
Rating utility...
Utility rating: 4/5
Overall score: 19
New best response found!

 ============== Processing context 2/3...
Generating response...
Assessing support...
Support ra

{'example1': {'query': 'What are the main ethical concerns in AI development?',
  'response': 'The main ethical concerns in AI development involve adhering to ethical principles, promoting fairness and transparency, and protecting human rights and values. This means ensuring that AI systems are developed in a way that does not discriminate against any group, is open and accountable in its operations, and respects the fundamental rights and dignity of individuals. It also includes addressing issues such as data privacy, algorithmic bias, and the potential for AI to have unintended consequences that could harm humans. A human-centered approach is crucial in addressing these ethical concerns, as it focuses on developing AI systems that are designed to serve and enhance human well-being rather than replace or undermine it.',
  'metrics': {'retrieval_needed': True,
   'documents_retrieved': 3,
   'relevant_documents': 3,
   'response_support_ratings': ['fully supported',
    'fully supporte

## 与传统 RAG 对比

In [27]:
def traditional_rag(query, vector_store, top_k=3):
    """
    Implements a traditional RAG approach for comparison.

    Args:
        query (str): User query
        vector_store (SimpleVectorStore): Vector store containing document chunks
        top_k (int): Number of documents to retrieve

    Returns:
        str: Generated response
    """
    print(f"\n=== Running traditional RAG for query: {query} ===\n")

    # Retrieve documents
    print("Retrieving documents...")
    query_embedding = create_embeddings(query)  # Create embeddings for the query
    results = vector_store.similarity_search(query_embedding, k=top_k)  # Search for similar documents
    print(f"Retrieved {len(results)} documents")

    # Combine contexts from retrieved documents
    contexts = [result["text"] for result in results]  # Extract text from results
    combined_context = "\n\n".join(contexts)  # Combine texts into a single context

    # Generate response using the combined context
    print("Generating response...")
    response = generate_response(query, combined_context)  # Generate response based on the combined context

    return response

In [28]:
def evaluate_rag_approaches(pdf_path, test_queries, reference_answers=None):
    """
    Compare Self-RAG with traditional RAG.

    Args:
        pdf_path (str): Path to the document
        test_queries (List[str]): List of test queries
        reference_answers (List[str], optional): Reference answers for evaluation

    Returns:
        dict: Evaluation results
    """
    print("=== Evaluating RAG Approaches ===")

    # Process document to create a vector store
    vector_store = process_document(pdf_path)

    results = []

    for i, query in enumerate(test_queries):
        print(f"\nProcessing query {i+1}: {query}")

        # Run Self-RAG
        self_rag_result = self_rag(query, vector_store)  # Get response from Self-RAG
        self_rag_response = self_rag_result["response"]

        # Run traditional RAG
        trad_rag_response = traditional_rag(query, vector_store)  # Get response from traditional RAG

        # Compare results if reference answer is available
        reference = reference_answers[i] if reference_answers and i < len(reference_answers) else None
        comparison = compare_responses(query, self_rag_response, trad_rag_response, reference)  # Compare responses

        results.append({
            "query": query,
            "self_rag_response": self_rag_response,
            "traditional_rag_response": trad_rag_response,
            "reference_answer": reference,
            "comparison": comparison,
            "self_rag_metrics": self_rag_result["metrics"]
        })

    # Generate overall analysis
    overall_analysis = generate_overall_analysis(results)

    return {
        "results": results,
        "overall_analysis": overall_analysis
    }

In [30]:
def compare_responses(query, self_rag_response, trad_rag_response, reference=None):
    """
    Compare responses from Self-RAG and traditional RAG.

    Args:
        query (str): User query
        self_rag_response (str): Response from Self-RAG
        trad_rag_response (str): Response from traditional RAG
        reference (str, optional): Reference answer

    Returns:
        str: Comparison analysis
    """
    system_prompt = """You are an expert evaluator of RAG systems. Your task is to compare responses from two different RAG approaches:
1. Self-RAG: A dynamic approach that decides if retrieval is needed and evaluates information relevance and response quality
2. Traditional RAG: Always retrieves documents and uses them to generate a response

Compare the responses based on:
- Relevance to the query
- Factual correctness
- Completeness and informativeness
- Conciseness and focus"""

    user_prompt = f"""Query: {query}

Response from Self-RAG:
{self_rag_response}

Response from Traditional RAG:
{trad_rag_response}
"""

    if reference:
        user_prompt += f"""
Reference Answer (for factual checking):
{reference}
"""

    user_prompt += """
Compare these responses and explain which one is better and why.
Focus on accuracy, relevance, completeness, and quality.
"""

    response = client.chat.completions.create(
        model=model_name,  # Using a stronger model for evaluation
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0
    )

    return response.choices[0].message.content

In [31]:
def generate_overall_analysis(results):
    """
    Generate an overall analysis of Self-RAG vs traditional RAG.

    Args:
        results (List[Dict]): Results from evaluate_rag_approaches

    Returns:
        str: Overall analysis
    """
    system_prompt = """You are an expert evaluator of RAG systems. Your task is to provide an overall analysis comparing
    Self-RAG and Traditional RAG based on multiple test queries.

    Focus your analysis on:
    1. When Self-RAG performs better and why
    2. When Traditional RAG performs better and why
    3. The impact of dynamic retrieval decisions in Self-RAG
    4. The value of relevance and support evaluation in Self-RAG
    5. Overall recommendations on which approach to use for different types of queries"""

    # Prepare a summary of the individual comparisons
    comparisons_summary = ""
    for i, result in enumerate(results):
        comparisons_summary += f"Query {i+1}: {result['query']}\n"
        comparisons_summary += f"Self-RAG metrics: Retrieval needed: {result['self_rag_metrics']['retrieval_needed']}, "
        comparisons_summary += f"Relevant docs: {result['self_rag_metrics']['relevant_documents']}/{result['self_rag_metrics']['documents_retrieved']}\n"
        comparisons_summary += f"Comparison summary: {result['comparison'][:200]}...\n\n"

        user_prompt = f"""Based on the following comparison results from {len(results)} test queries, please provide an overall analysis of
    Self-RAG versus Traditional RAG:

    {comparisons_summary}

    Please provide your comprehensive analysis.
    """

    response = client.chat.completions.create(
        model=model_name,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0
    )

    return response.choices[0].message.content

## 评估

In [32]:
# Path to the AI information document
pdf_path = "./drive/MyDrive/colab_data/AI_Information.pdf"

# Define test queries covering different query types to test Self-RAG's adaptive retrieval
test_queries = [
    "What are the main ethical concerns in AI development?",        # Document-focused query
    # "How does explainable AI improve trust in AI systems?",         # Document-focused query
    # "Write a poem about artificial intelligence",                   # Creative query, doesn't need retrieval
    # "Will superintelligent AI lead to human obsolescence?"          # Speculative query, partial retrieval needed
]

# Reference answers for more objective evaluation
reference_answers = [
    "The main ethical concerns in AI development include bias and fairness, privacy, transparency, accountability, safety, and the potential for misuse or harmful applications.",
    # "Explainable AI improves trust by making AI decision-making processes transparent and understandable to users, helping them verify fairness, identify potential biases, and better understand AI limitations.",
    # "A quality poem about artificial intelligence should creatively explore themes of AI's capabilities, limitations, relationship with humanity, potential futures, or philosophical questions about consciousness and intelligence.",
    # "Views on superintelligent AI's impact on human relevance vary widely. Some experts warn of potential risks if AI surpasses human capabilities across domains, possibly leading to economic displacement or loss of human agency. Others argue humans will remain relevant through complementary skills, emotional intelligence, and by defining AI's purpose. Most experts agree that thoughtful governance and human-centered design are essential regardless of the outcome."
]

# Run the evaluation comparing Self-RAG with traditional RAG approaches
evaluation_results = evaluate_rag_approaches(
    pdf_path=pdf_path,                  # Source document containing AI information
    test_queries=test_queries,          # List of AI-related test queries
    reference_answers=reference_answers  # Ground truth answers for evaluation
)

# Print the overall comparative analysis
print("\n=== OVERALL ANALYSIS ===\n")
print(evaluation_results["overall_analysis"])

=== Evaluating RAG Approaches ===
Extracting text from PDF...
Chunking text...
Created 42 text chunks
Creating embeddings for chunks...
Added 42 chunks to the vector store

Processing query 1: What are the main ethical concerns in AI development?

=== Starting Self-RAG for query: What are the main ethical concerns in AI development? ===

Step 1: Determining if retrieval is necessary...
Retrieval needed: True

Step 2: Retrieving relevant documents...
Retrieved 3 documents

Step 3: Evaluating document relevance...
Document 1 relevance: relevant
Document 2 relevance: relevant
Document 3 relevance: relevant
Found 3 relevant documents

Step 4: Processing relevant contexts...

 ============== Processing context 1/3...
Generating response...
Assessing support...
Support rating: fully supported
Rating utility...
Utility rating: 4/5
Overall score: 19
New best response found!

 ============== Processing context 2/3...
Generating response...
Assessing support...
Support rating: fully supported
Ra